# Hosting Detectron2 model on Sagemaker Inference endpoint

In this notebook we'll package previously trained model into PyTorch Serving container and deploy it on Sagemaker. First, let's review serving container. There are two key difference comparing to training container:
- we are using different base container provided by Sagemaker;
- we need to start Web server (refer to ENTRYPOINT command).

## Compiling Serving Container

In [ ]:
! pygmentize -l docker Dockerfile.serving

As in case of training image, we'll need to build and push container to AWS ECR. Before this, we'll need to loging to shared Sagemaker ECR and your local ECR

In [ ]:
# loging to Sagemaker ECR with Deep Learning Containers
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-2.amazonaws.com
# loging to your private ECR
!aws ecr get-login-password --region us-east-2 | docker login --username AWS --password-stdin 553020858742.dkr.ecr.us-east-2.amazonaws.com

Now, let's build and push container using follow command. Note, that here we supply non-default Dockerfile.

In [ ]:
! ./build_and_push.sh d2-sm-coco-serving latest Dockerfile.serving

## Preparing test data

We'll be using coco2017 validation dataset. To simplify working with it, let's install locally Pycoco package

In [ ]:
!pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
!pip install -U scikit-image

Now, let's download Coco2017 validation dataset

In [ ]:
data_dir = "../datasets/coco/" # folder where data will be saved
dataset  = "val2017"

In [ ]:
! mkdir -p {data_dir}{dataset}
! wget http://images.cocodataset.org/zips/val2017.zip -P {data_dir}
! unzip {data_dir}/val2017.zip -d {data_dir}

In [ ]:
! wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -P {data_dir}
! unzip {data_dir}/annotations_trainval2017.zip -d {data_dir}{dataset}

Let's get a random image ...

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)

annFile='{}{}/annotations/instances_{}.json'.format(data_dir,dataset, dataset)
coco=COCO(annFile)

# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['person','dog']);
imgIds = coco.getImgIds(catIds=catIds);
imgId = imgIds[np.random.randint(len(imgIds))]
image_instance = coco.loadImgs(imgId)[0]
image_np = io.imread(image_instance['coco_url'])    

In [ ]:
plt.axis('off')
plt.imshow(image_np)
plt.show()

# Testing inference script locally

Let's first check what inference script we'll deploy:

In [ ]:
!pip install sagemaker-inference

In [ ]:
!pygmentize container_serving/predict_coco.py

To test inference pipeline locally, you can run your `container_serving/predict_coco.py` locally (only code in __main__ guard will be executed). You'll need to have Detectron2 and number other packages locally installed to test it.

Make sure that you pass correct --model-dir argument.

In [ ]:
!python container_serving/predict_coco.py --image container_serving/coco_sample.jpg --model-dir ../trained_model

# Deploying Inference Endpoint

Below is some initial imports and configuration.

In [3]:
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [4]:
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session() # can use LocalSession() to run container locally

bucket = sess.default_bucket()
region = "us-east-2"
account = sess.boto_session.client('sts').get_caller_identity()['Account']
prefix_input = 'detectron2-input'
prefix_output = 'detectron2-ouput'

## Define parameters of your container

In [5]:
container_serving = "d2-sm-coco-serving" # your container name
tag = "latest" # you can have several version of container available
image = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account, region, container_serving, tag)

print("Following container will be used for hosting: ",image)

Following container will be used for hosting:  553020858742.dkr.ecr.us-east-2.amazonaws.com/d2-sm-coco-serving:latest


## Deploy endpoint locally

As training on COCO2017 can be quite lenghty, we'll deploy our endpoint from model artifacts from already completed training jobs. Please review your training jobs, and find one which succesffuly completed. Then, copy model artifact S3 URI and.  pass it to `model_data` argument below.

In [4]:
from sagemaker.pytorch import PyTorchModel, PyTorch, PyTorchPredictor
from sagemaker import Model

# model = Model(model_data="s3://sagemaker-us-east-2-553020858742/detectron2-model/model_R_50_FPN_1x.tar.gz",
#               role=role,
#               image=image)

model = PyTorchModel(
#                     model_data="s3://sagemaker-us-east-2-553020858742/detectron2-model/model.tar.gz", #default D2 model
                     model_data="s3://sagemaker-us-east-2-553020858742/detectron2-model/model_R_50_FPN_1x.tar.gz", # from training job
                     role=role,
                     entry_point="predict_coco.py", source_dir="container_serving",
                     framework_version="1.4", py_version="3.6",
                     image=image)

In [ ]:
predictor = model.deploy(
                         instance_type = 'local_gpu',
                         initial_instance_count=1,
                         endpoint_name=f"{container_serving}-{tag}", # define a unqie endpoint name; if ommited, Sagemaker will generate it based on used container
                         tags=[{"Key":"image", "Value":f"{container_serving}:{tag}"}], 
                         wait=False
                         )

In [25]:
import requests
import cv2

url = 'http://172.18.0.2:8080/invocations'
content_type = 'image/jpeg'
headers = {'content-type': content_type}
payload = open('coco_sample.jpg', 'rb')

response = requests.post(url, data=payload, headers=headers)
print(response.content)

b'\x80\x03}q\x00X\t\x00\x00\x00instancesq\x01cdetectron2.structures.instances\nInstances\nq\x02)\x81q\x03}q\x04(X\x0b\x00\x00\x00_image_sizeq\x05M\xe0\x01M\x80\x02\x86q\x06X\x07\x00\x00\x00_fieldsq\x07}q\x08(X\n\x00\x00\x00pred_boxesq\tcdetectron2.structures.boxes\nBoxes\nq\n)\x81q\x0b}q\x0cX\x06\x00\x00\x00tensorq\rctorch._utils\n_rebuild_tensor_v2\nq\x0e(ctorch.storage\n_load_from_bytes\nq\x0fB.\x02\x00\x00\x80\x02\x8a\nl\xfc\x9cF\xf9 j\xa8P\x19.\x80\x02M\xe9\x03.\x80\x02}q\x00(X\x10\x00\x00\x00protocol_versionq\x01M\xe9\x03X\r\x00\x00\x00little_endianq\x02\x88X\n\x00\x00\x00type_sizesq\x03}q\x04(X\x05\x00\x00\x00shortq\x05K\x02X\x03\x00\x00\x00intq\x06K\x04X\x04\x00\x00\x00longq\x07K\x04uu.\x80\x02(X\x07\x00\x00\x00storageq\x00ctorch\nFloatStorage\nq\x01X\x0e\x00\x00\x0094519452522384q\x02X\x06\x00\x00\x00cuda:0q\x03KLNtq\x04Q.\x80\x02]q\x00X\x0e\x00\x00\x0094519452522384q\x01a.L\x00\x00\x00\x00\x00\x00\x00k\xfc\xb0Cx\xa2.C\xf3\xfc\xbaC=\xf3XC\xb7\xe1?B\xb3y\xb7Ca\x13\x91B\x1f\xa4\x

In [ ]:
type(response.content)
import pickle
import detectron2 

p = pickle.loads(response.content)

## Deploy remote endpoint

To process inference data when we are sending it over internet, we need to have two customer ser/deser methods.

In [6]:
from sagemaker.pytorch import PyTorchModel, PyTorch, PyTorchPredictor
from sagemaker.estimator import Estimator, Model

remote_model = PyTorchModel(
                     model_data="s3://sagemaker-us-east-2-553020858742/detectron2-model/model_R_50_FPN_1x.tar.gz", # from training job
                     role=role,
                     entry_point="predict_coco.py", source_dir="container_serving",
                     framework_version="1.4", py_version="3.6",
                     image=image)

In [7]:
remote_predictor = remote_model.deploy(
                         instance_type='ml.p3.16xlarge', 
                         initial_instance_count=1,
                         endpoint_name=f"{container_serving}-{tag}", # define a unqie endpoint name; if ommited, Sagemaker will generate it based on used container
                         tags=[{"Key":"image", "Value":f"{container_serving}:{tag}"}], 
                         wait=False
                         )

In [ ]:
import boto3

client = boto3.client('sagemaker-runtime')

content_type = 'image/jpeg'
headers = {'content-type': content_type}
payload = open('coco_sample.jpg', 'rb')

client.invoke_endpoint(
    EndpointName=remote_predictor.endpoint,
    Body=payload,
    ContentType=content_type,
#    Accept = ""
)

In [ ]:
predictor.delete_endpoint()